# Notebook 04: RAG con Routing - Agente Multi-Dominio que Clasifica y Delega

**Modulo 05 - RAG (Retrieval-Augmented Generation) | AI Engineering - Henry**

---

## Contexto

En los notebooks anteriores construimos los fundamentos:
- **Notebook 01**: RAG basico con embeddings y retrieval sobre una sola base de conocimiento.
- **Notebook 02**: Chunking strategies y optimizacion de la calidad de retrieval.
- **Notebook 03**: RAG con Prompt Chaining - encadenando pasos de razonamiento.

Ahora damos el siguiente salto: **Routing**. Un sistema RAG que primero **clasifica la intencion** del usuario y luego **delega a un agente especializado** segun el dominio detectado.

## Objetivos de aprendizaje

1. Entender **por que** un sistema multi-dominio necesita routing en lugar de un solo retriever.
2. Implementar un **clasificador de intencion** usando LLM con structured output.
3. Construir **agentes RAG especializados** por dominio (Productos y Tecnica).
4. Orquestar todo con **LangGraph**: nodos, edges condicionales y estado compartido.
5. Evaluar el impacto del **umbral de confianza** en las decisiones de routing.
6. Comparar routing vs single retriever para entender las ventajas.

## Empresa: NovaTech Solutions

Trabajamos con dos dominios de conocimiento:
- **PRODUCTOS**: Informacion de productos, precios, features, planes (Analytics Pro, DataSync, AI Assistant).
- **TECNICA**: Operaciones tecnicas, deploys, troubleshooting, monitoreo, seguridad.

---

## 1. Por que Routing para RAG?

### El problema del retriever unico

Imagina que tienes una empresa con documentacion de **productos** (precios, features, planes) y documentacion **tecnica** (deploys, Kubernetes, troubleshooting). Si indexas todo junto en una sola coleccion:

- La query "cuanto cuesta el plan Enterprise?" podria traer chunks de troubleshooting que mencionan "Enterprise" por contexto.
- La query "como hago rollback?" podria traer chunks de producto que mencionan "rollback automatico" en features.
- El retriever no tiene **contexto semantico del dominio** -- solo busca similitud vectorial.

### La solucion: clasificar y delegar

```
                        +---------------------+
                        |    User Query        |
                        +----------+----------+
                                   |
                                   v
                        +---------------------+
                        | Intent Classifier   |
                        | (LLM + Structured   |
                        |  Output)            |
                        +----+------+----+----+
                             |      |    |
                   PRODUCTOS |      |    | UNKNOWN
                             |      |    |
                             v      |    v
                  +----------+--+   |  +-+----------+
                  | Product RAG |   |  |  Fallback  |
                  | Agent       |   |  |  Agent     |
                  +----------+--+   |  +-+----------+
                             |      |    |
                             |  TECNICA  |
                             |      |    |
                             |      v    |
                             | +----+------+
                             | | Tech RAG  |
                             | | Agent     |
                             | +----+------+
                             |      |    |
                             v      v    v
                        +---------------------+
                        |   Respuesta Final   |
                        +---------------------+
```

### Ventajas del routing

| Aspecto | Single Retriever | Routing |
|---------|-----------------|----------|
| Precision de retrieval | Contaminacion entre dominios | Busqueda aislada por dominio |
| Prompt del LLM | Generico, sin contexto de dominio | Especializado por dominio |
| Escalabilidad | Agregar docs degrada todo | Agregar un dominio nuevo es independiente |
| Trazabilidad | No sabes por que eligio ciertos docs | Sabes que dominio y por que |
| Costo | Siempre busca en todo | Solo busca en el dominio relevante |

### Cuando usar routing vs single retriever

- **Single retriever**: Documentacion homogenea, un solo tema, base pequena (<100 docs).
- **Routing**: Multiples dominios, tipos de pregunta distintos, necesidad de respuestas especializadas.

---

## 2. Arquitectura del Sistema

### Grafo completo con LangGraph

```
+-------------------------------------------------------------------+
|                         RouterState                               |
|  query | intent | retrieved_docs | response | route | steps_trace |
+-------------------------------------------------------------------+
                              |
                              v
                    [classify_intent]
                    Nodo de entrada
                    - Recibe query
                    - LLM clasifica intent
                    - Retorna: intent, confidence, reasoning
                              |
                    (conditional edge)
                    route_by_intent()
                              |
            +-----------------+-----------------+
            |                 |                 |
            v                 v                 v
   [product_agent]    [tech_agent]       [fallback]
   confidence >= 0.6  confidence >= 0.6  confidence < 0.6
   intent=PRODUCTOS   intent=TECNICA     o intent=UNKNOWN
   - Retrieve from    - Retrieve from    - Mensaje de
     col. productos     col. tecnica       clarificacion
   - Prompt especial  - Prompt especial  - Sin retrieval
            |                 |                 |
            v                 v                 v
          [END]             [END]             [END]
```

### Flujo del estado

1. El estado se inicializa con la `query` del usuario.
2. `classify_intent` llena `intent` (con confidence y reasoning).
3. `route_by_intent` evalua confidence + intent para decidir la ruta.
4. El agente elegido llena `retrieved_docs`, `response` y `route_taken`.
5. Cada nodo agrega su nombre a `steps_trace` para trazabilidad.

### Routing basado en confianza

El clasificador no solo dice "PRODUCTOS" o "TECNICA": tambien da un **score de confianza** (0.0 a 1.0). Si la confianza es baja (<0.6), incluso si clasifico como PRODUCTOS, preferimos ir al fallback. Esto evita respuestas incorrectas cuando la query es ambigua.

---

## 3. Setup

In [ ]:
# =============================================================================
# Instalacion de dependencias (ejecutar una vez)
# =============================================================================
# %pip install openai chromadb langchain langchain-openai langchain-core langgraph pydantic python-dotenv matplotlib pandas --quiet

In [ ]:
# =============================================================================
# Imports principales
# =============================================================================
import os
import json
from typing import Optional, Literal, TypedDict
from pathlib import Path

# LLM y embeddings
from openai import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage

# Vector store
import chromadb
from chromadb.config import Settings

# Text splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter

# LangGraph
from langgraph.graph import StateGraph, END

# Pydantic para structured output
from pydantic import BaseModel, Field

# Visualizacion y datos
import matplotlib.pyplot as plt
import pandas as pd

# Variables de entorno
from dotenv import load_dotenv

print("Imports completados exitosamente.")

In [ ]:
# =============================================================================
# Cargar variables de entorno e inicializar LLM
# =============================================================================
load_dotenv()

# Verificar que la API key esta configurada
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY no encontrada. Verifica tu archivo .env"

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,  # Deterministico para clasificacion
)

# Inicializar embeddings
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Inicializar cliente OpenAI directo (para structured output)
client = OpenAI()

print("LLM inicializado: gpt-4o-mini")
print("Embeddings: text-embedding-3-small")
print("Setup completado.")

---

## 4. Preparar Bases de Conocimiento por Dominio

La clave del routing es que cada dominio tiene su **propia coleccion** en ChromaDB. Esto significa:
- Los embeddings de productos solo compiten con otros embeddings de productos.
- Los embeddings tecnicos solo compiten con otros embeddings tecnicos.
- No hay contaminacion cruzada en el retrieval.

Vamos a:
1. Cargar cada archivo `.md` de la carpeta `data/`.
2. Dividirlo en chunks con `RecursiveCharacterTextSplitter`.
3. Indexar cada conjunto de chunks en su propia coleccion ChromaDB.

In [ ]:
# =============================================================================
# Cargar documentos fuente
# =============================================================================

# Rutas a las bases de conocimiento
DATA_DIR = Path("../data")

ruta_productos = DATA_DIR / "base_conocimiento_productos.md"
ruta_tecnica = DATA_DIR / "base_conocimiento_tecnica.md"

# Leer contenido de cada archivo
with open(ruta_productos, "r", encoding="utf-8") as f:
    contenido_productos = f.read()

with open(ruta_tecnica, "r", encoding="utf-8") as f:
    contenido_tecnica = f.read()

print(f"Documento PRODUCTOS cargado: {len(contenido_productos):,} caracteres")
print(f"Documento TECNICA cargado:   {len(contenido_tecnica):,} caracteres")

In [ ]:
# =============================================================================
# Chunking: dividir documentos en fragmentos manejables
# =============================================================================

# Configuracion del splitter
# - chunk_size=500: fragmentos de ~500 caracteres (balanceado para precision)
# - chunk_overlap=100: solapamiento para no perder contexto en bordes
# - Separadores: priorizamos cortar por secciones markdown, luego parrafos, luego oraciones
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n## ", "\n### ", "\n\n", "\n", ". ", " "],
    length_function=len,
)

# Dividir cada documento
chunks_productos = text_splitter.split_text(contenido_productos)
chunks_tecnica = text_splitter.split_text(contenido_tecnica)

print(f"Chunks PRODUCTOS: {len(chunks_productos)}")
print(f"Chunks TECNICA:   {len(chunks_tecnica)}")
print(f"\n--- Ejemplo chunk PRODUCTOS (chunk 0) ---")
print(chunks_productos[0][:300] + "...")
print(f"\n--- Ejemplo chunk TECNICA (chunk 0) ---")
print(chunks_tecnica[0][:300] + "...")

In [ ]:
# =============================================================================
# Indexar en ChromaDB: una coleccion por dominio
# =============================================================================

# Inicializar cliente ChromaDB (en memoria para este notebook)
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))

def crear_coleccion(nombre: str, chunks: list[str]) -> chromadb.Collection:
    """Crea una coleccion en ChromaDB e indexa los chunks con embeddings de OpenAI."""
    
    # Eliminar coleccion si ya existe (para re-ejecucion limpia)
    try:
        chroma_client.delete_collection(nombre)
    except Exception:
        pass
    
    # Crear coleccion nueva
    coleccion = chroma_client.create_collection(
        name=nombre,
        metadata={"hnsw:space": "cosine"}  # Distancia coseno para similitud
    )
    
    # Generar embeddings para cada chunk
    embeddings = embeddings_model.embed_documents(chunks)
    
    # Agregar documentos a la coleccion
    coleccion.add(
        documents=chunks,
        embeddings=embeddings,
        ids=[f"{nombre}_chunk_{i}" for i in range(len(chunks))],
        metadatas=[{"dominio": nombre, "chunk_index": i} for i in range(len(chunks))],
    )
    
    return coleccion

# Crear colecciones por dominio
col_productos = crear_coleccion("productos", chunks_productos)
col_tecnica = crear_coleccion("tecnica", chunks_tecnica)

print(f"Coleccion 'productos' creada: {col_productos.count()} documentos indexados")
print(f"Coleccion 'tecnica' creada:   {col_tecnica.count()} documentos indexados")
print("\nBases de conocimiento listas para retrieval.")

---

## 5. Definir Esquemas

Usamos **Pydantic** para definir estructuras de datos rigidas. Esto nos da:
- Validacion automatica de tipos y rangos.
- Structured output del LLM (el modelo responde directamente en este formato).
- Documentacion implicita del contrato de datos.

In [ ]:
# =============================================================================
# Esquema: Clasificacion de intencion
# =============================================================================

class IntentClassification(BaseModel):
    """Resultado de la clasificacion de intencion del usuario."""
    intent: Literal["PRODUCTOS", "TECNICA", "UNKNOWN"] = Field(
        description="Dominio clasificado de la consulta del usuario"
    )
    confidence: float = Field(
        ge=0.0, le=1.0,
        description="Nivel de confianza de la clasificacion (0.0 a 1.0)"
    )
    reasoning: str = Field(
        description="Explicacion breve de por que se clasifico en este dominio"
    )


# =============================================================================
# Esquema: Respuesta del sistema RAG
# =============================================================================

class RAGResponse(BaseModel):
    """Respuesta generada por un agente RAG especializado."""
    answer: str = Field(
        description="Respuesta completa a la consulta del usuario"
    )
    sources: list[str] = Field(
        description="Fragmentos de documentacion usados como fuente"
    )
    confidence: float = Field(
        ge=0.0, le=1.0,
        description="Confianza del agente en la calidad de su respuesta"
    )
    domain: str = Field(
        description="Dominio que genero la respuesta"
    )
    follow_up: str = Field(
        description="Pregunta de seguimiento sugerida para el usuario"
    )


print("Esquema IntentClassification:")
print(json.dumps(IntentClassification.model_json_schema(), indent=2, ensure_ascii=False)[:500])
print("\nEsquema RAGResponse:")
print(json.dumps(RAGResponse.model_json_schema(), indent=2, ensure_ascii=False)[:500])

---

## 6. Definir Estado del Grafo

El estado es el **objeto compartido** que fluye entre todos los nodos del grafo. Cada nodo lee y escribe en este estado. Es el equivalente a la "memoria de trabajo" del sistema.

In [ ]:
# =============================================================================
# Estado del grafo: lo que todos los nodos comparten
# =============================================================================

class RouterState(TypedDict):
    """Estado compartido entre todos los nodos del grafo de routing."""
    query: str                                    # Consulta original del usuario
    intent: Optional[IntentClassification]        # Resultado de la clasificacion
    retrieved_docs: list[str]                     # Documentos recuperados por el agente
    response: Optional[RAGResponse]               # Respuesta final generada
    route_taken: str                              # Nombre de la ruta elegida
    steps_trace: list[str]                        # Traza de nodos visitados


print("RouterState definido con campos:")
for campo, tipo in RouterState.__annotations__.items():
    print(f"  - {campo}: {tipo}")

---

## 7. Nodo 1: Clasificador de Intencion

Este es el **cerebro del routing**. Recibe la query del usuario y decide a que dominio pertenece.

### Como funciona
1. Toma la query del estado.
2. Envia un prompt al LLM con **ejemplos** de cada categoria (few-shot).
3. Usa **structured output** para forzar que el LLM responda en formato `IntentClassification`.
4. Retorna el estado actualizado con la clasificacion.

### Nota sobre el system prompt
El prompt incluye ejemplos concretos de cada dominio para guiar al modelo. Esto es critico: sin ejemplos, el modelo tiende a clasificar demasiadas queries como UNKNOWN.

In [ ]:
# =============================================================================
# Nodo 1: Clasificador de intencion (intent classifier)
# =============================================================================

CLASIFICADOR_SYSTEM_PROMPT = """Eres un clasificador de intenciones para NovaTech Solutions.
Tu trabajo es determinar si la consulta del usuario pertenece a uno de estos dominios:

PRODUCTOS:
- Preguntas sobre precios, planes, features de productos (Analytics Pro, DataSync, AI Assistant)
- Comparaciones entre planes (Starter, Professional, Enterprise, Growth, Scale, Business)
- Limites, requisitos tecnicos de productos
- Politicas de soporte, SLAs, reembolsos
- Preguntas frecuentes sobre uso de productos
Ejemplos: "cuanto cuesta el plan Enterprise?", "que incluye DataSync Growth?",
"cual es el SLA de Analytics Pro?", "como migro entre planes?"

TECNICA:
- Procedimientos de deploy, rollback, CI/CD
- Troubleshooting de errores y problemas operativos
- Arquitectura de infraestructura, microservicios, Kubernetes
- Monitoreo, alertas, metricas (Datadog, PagerDuty, Sentry)
- Seguridad, autenticacion, gestion de secretos
- On-call, respuesta a incidentes
Ejemplos: "como hago rollback en produccion?", "que hago si analytics-engine no responde?",
"como roto secretos en AWS?", "cual es el proceso de deploy de emergencia?"

UNKNOWN:
- Preguntas que no tienen relacion con NovaTech Solutions
- Temas personales, recetas, deportes, etc.
- Si no estas seguro, clasifica como UNKNOWN con confidence baja

IMPORTANTE:
- Asigna un score de confidence entre 0.0 y 1.0
- Si la query es ambigua (podria ser de cualquier dominio), baja la confidence
- Siempre explica tu razonamiento brevemente
"""


def classify_intent(state: RouterState) -> RouterState:
    """Clasifica la intencion de la query del usuario usando LLM con structured output."""
    
    query = state["query"]
    
    # Llamada al LLM con structured output (response_format de OpenAI)
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        temperature=0.0,
        messages=[
            {"role": "system", "content": CLASIFICADOR_SYSTEM_PROMPT},
            {"role": "user", "content": f"Clasifica esta consulta: {query}"},
        ],
        response_format=IntentClassification,
    )
    
    # Extraer la clasificacion parseada
    clasificacion = response.choices[0].message.parsed
    
    # Imprimir resultado para visibilidad
    print(f"[CLASIFICADOR] Query: '{query}'")
    print(f"  Intent:     {clasificacion.intent}")
    print(f"  Confidence: {clasificacion.confidence}")
    print(f"  Reasoning:  {clasificacion.reasoning}")
    
    # Actualizar estado
    return {
        **state,
        "intent": clasificacion,
        "steps_trace": state.get("steps_trace", []) + ["classify_intent"],
    }


print("Nodo classify_intent definido.")

---

## 8. Nodo 2: Agente RAG de Productos

Este agente se activa cuando el clasificador detecta intent=PRODUCTOS con alta confianza.

Su especialidad:
- Busca **solo** en la coleccion `productos`.
- Tiene un system prompt optimizado para responder sobre precios, planes, features.
- Conoce la estructura de productos de NovaTech (Analytics Pro, DataSync, AI Assistant).

In [ ]:
# =============================================================================
# Funcion auxiliar: buscar documentos en una coleccion ChromaDB
# =============================================================================

def buscar_documentos(
    coleccion: chromadb.Collection,
    query: str,
    n_results: int = 3
) -> list[str]:
    """Busca los documentos mas relevantes en una coleccion ChromaDB."""
    
    # Generar embedding de la query
    query_embedding = embeddings_model.embed_query(query)
    
    # Buscar en la coleccion
    resultados = coleccion.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
    )
    
    # Extraer documentos (ChromaDB retorna listas anidadas)
    documentos = resultados["documents"][0] if resultados["documents"] else []
    return documentos


print("Funcion buscar_documentos definida.")

In [ ]:
# =============================================================================
# Nodo 2: Agente RAG de Productos
# =============================================================================

PRODUCTO_SYSTEM_PROMPT = """Eres un agente experto en productos de NovaTech Solutions.
Tu especialidad es responder preguntas sobre:
- NovaTech Analytics Pro: plataforma de business intelligence
- NovaTech DataSync: herramienta de ETL
- NovaTech AI Assistant: chatbot empresarial con IA

Usa UNICAMENTE la informacion proporcionada en el contexto para responder.
Si el contexto no contiene la informacion, dilo honestamente.
Responde en espanol de forma profesional y concisa.
Cuando menciones precios, se preciso con los valores y condiciones.
Sugiere una pregunta de seguimiento relevante para el usuario.
"""


def product_rag_agent(state: RouterState) -> RouterState:
    """Agente RAG especializado en informacion de productos NovaTech."""
    
    query = state["query"]
    
    # Paso 1: Recuperar documentos relevantes de la coleccion de productos
    docs = buscar_documentos(col_productos, query, n_results=3)
    
    print(f"[AGENTE PRODUCTOS] Documentos recuperados: {len(docs)}")
    for i, doc in enumerate(docs):
        print(f"  Doc {i}: {doc[:100]}...")
    
    # Paso 2: Construir contexto para el LLM
    contexto = "\n\n---\n\n".join(docs)
    
    # Paso 3: Generar respuesta con structured output
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        temperature=0.2,
        messages=[
            {"role": "system", "content": PRODUCTO_SYSTEM_PROMPT},
            {"role": "user", "content": (
                f"CONTEXTO:\n{contexto}\n\n"
                f"PREGUNTA DEL USUARIO:\n{query}"
            )},
        ],
        response_format=RAGResponse,
    )
    
    rag_response = response.choices[0].message.parsed
    # Asegurar que el dominio este correcto
    rag_response.domain = "PRODUCTOS"
    
    print(f"  Respuesta generada (confidence: {rag_response.confidence})")
    
    # Actualizar estado
    return {
        **state,
        "retrieved_docs": docs,
        "response": rag_response,
        "route_taken": "product_agent",
        "steps_trace": state.get("steps_trace", []) + ["product_rag_agent"],
    }


print("Nodo product_rag_agent definido.")

---

## 9. Nodo 3: Agente RAG Tecnico

Este agente maneja todas las consultas de operaciones, infraestructura y troubleshooting.

Su especialidad:
- Busca **solo** en la coleccion `tecnica`.
- Tiene un system prompt optimizado para procedimientos operativos.
- Conoce la arquitectura de microservicios, Kubernetes, CI/CD de NovaTech.

In [ ]:
# =============================================================================
# Nodo 3: Agente RAG Tecnico
# =============================================================================

TECNICA_SYSTEM_PROMPT = """Eres un agente experto en operaciones tecnicas de NovaTech Solutions.
Tu especialidad es responder preguntas sobre:
- Arquitectura de microservicios (12 servicios en Kubernetes/EKS)
- Procedimientos de deploy y rollback
- Troubleshooting de errores comunes
- Monitoreo con Datadog, alertas con PagerDuty
- Gestion de secretos con AWS Secrets Manager
- Seguridad, autenticacion, compliance
- Procedimientos de on-call y respuesta a incidentes

Usa UNICAMENTE la informacion proporcionada en el contexto para responder.
Si el contexto no contiene la informacion, dilo honestamente.
Responde en espanol de forma tecnica pero clara.
Cuando menciones comandos, incluyelos con formato de codigo.
Sugiere una pregunta de seguimiento relevante para el usuario.
"""


def tech_rag_agent(state: RouterState) -> RouterState:
    """Agente RAG especializado en operaciones tecnicas de NovaTech."""
    
    query = state["query"]
    
    # Paso 1: Recuperar documentos relevantes de la coleccion tecnica
    docs = buscar_documentos(col_tecnica, query, n_results=3)
    
    print(f"[AGENTE TECNICO] Documentos recuperados: {len(docs)}")
    for i, doc in enumerate(docs):
        print(f"  Doc {i}: {doc[:100]}...")
    
    # Paso 2: Construir contexto para el LLM
    contexto = "\n\n---\n\n".join(docs)
    
    # Paso 3: Generar respuesta con structured output
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        temperature=0.2,
        messages=[
            {"role": "system", "content": TECNICA_SYSTEM_PROMPT},
            {"role": "user", "content": (
                f"CONTEXTO:\n{contexto}\n\n"
                f"PREGUNTA DEL USUARIO:\n{query}"
            )},
        ],
        response_format=RAGResponse,
    )
    
    rag_response = response.choices[0].message.parsed
    # Asegurar que el dominio este correcto
    rag_response.domain = "TECNICA"
    
    print(f"  Respuesta generada (confidence: {rag_response.confidence})")
    
    # Actualizar estado
    return {
        **state,
        "retrieved_docs": docs,
        "response": rag_response,
        "route_taken": "tech_agent",
        "steps_trace": state.get("steps_trace", []) + ["tech_rag_agent"],
    }


print("Nodo tech_rag_agent definido.")

---

## 10. Nodo 4: Agente Fallback

Se activa cuando:
- La clasificacion es UNKNOWN (query fuera de dominio).
- La confianza es menor al umbral (query ambigua).

**No hace retrieval** -- simplemente genera un mensaje educado pidiendo clarificacion y sugiriendo los dominios disponibles.

In [ ]:
# =============================================================================
# Nodo 4: Agente Fallback
# =============================================================================

def fallback_agent(state: RouterState) -> RouterState:
    """Agente fallback: responde cuando no se puede clasificar la consulta con confianza."""
    
    query = state["query"]
    intent = state.get("intent")
    
    # Construir mensaje de fallback
    if intent and intent.intent == "UNKNOWN":
        mensaje = (
            f"Lo siento, tu consulta '{query}' no parece estar relacionada con los "
            f"servicios de NovaTech Solutions. "
            f"Puedo ayudarte con:\n"
            f"- **Productos**: Precios, planes, features de Analytics Pro, DataSync y AI Assistant.\n"
            f"- **Tecnica**: Deploy, rollback, troubleshooting, monitoreo, seguridad.\n\n"
            f"Por favor, reformula tu pregunta dentro de estos dominios."
        )
    else:
        # Baja confianza en la clasificacion
        mensaje = (
            f"Tu consulta '{query}' es un poco ambigua y no estoy seguro de como "
            f"dirigirla correctamente. "
            f"Podrias especificar si tu pregunta es sobre:\n"
            f"- **Productos**: Precios, planes, features, SLAs.\n"
            f"- **Tecnica**: Deploy, errores, infraestructura, seguridad.\n\n"
            f"Asi podre darte una respuesta mas precisa."
        )
    
    # Crear respuesta de fallback
    rag_response = RAGResponse(
        answer=mensaje,
        sources=[],
        confidence=0.0,
        domain="FALLBACK",
        follow_up="Podrias reformular tu pregunta especificando si es sobre productos o temas tecnicos?",
    )
    
    print(f"[FALLBACK] Query: '{query}'")
    print(f"  Razon: {'fuera de dominio' if intent and intent.intent == 'UNKNOWN' else 'baja confianza'}")
    
    # Actualizar estado
    return {
        **state,
        "retrieved_docs": [],
        "response": rag_response,
        "route_taken": "fallback",
        "steps_trace": state.get("steps_trace", []) + ["fallback_agent"],
    }


print("Nodo fallback_agent definido.")

---

## 11. Funcion de Routing (Conditional Edges)

Esta funcion es el **decision maker** del grafo. No es un nodo -- es la funcion que LangGraph usa para decidir que edge seguir despues del clasificador.

### Logica de decision
```
if confidence >= 0.6 AND intent == "PRODUCTOS"  --> product_agent
if confidence >= 0.6 AND intent == "TECNICA"     --> tech_agent
else (baja confianza o UNKNOWN)                  --> fallback
```

### Por que 0.6 como umbral?

- **0.3 (muy bajo)**: Demasiadas queries dudosas se enrutan a agentes especializados. Riesgo de respuestas incorrectas.
- **0.6 (balanceado)**: Acepta clasificaciones razonablemente seguras. Queries ambiguas van a fallback.
- **0.8 (muy alto)**: Demasiadas queries validas caen en fallback. Frustracion del usuario.

En la seccion de ajuste de umbral experimentaremos con estos valores.

In [ ]:
# =============================================================================
# Funcion de routing: decide que edge seguir
# =============================================================================

# Umbral de confianza configurable (lo ajustaremos despues)
CONFIDENCE_THRESHOLD = 0.6


def route_by_intent(state: RouterState) -> str:
    """Decide la ruta basandose en el intent clasificado y su confianza.
    
    Returns:
        str: Nombre del nodo destino ("product_agent", "tech_agent", o "fallback")
    """
    
    intent = state.get("intent")
    
    # Si no hay clasificacion, ir a fallback
    if intent is None:
        print(f"[ROUTER] Sin clasificacion -> fallback")
        return "fallback"
    
    # Evaluar confianza y dominio
    if intent.confidence >= CONFIDENCE_THRESHOLD and intent.intent == "PRODUCTOS":
        print(f"[ROUTER] PRODUCTOS (confidence={intent.confidence:.2f}) -> product_agent")
        return "product_agent"
    
    elif intent.confidence >= CONFIDENCE_THRESHOLD and intent.intent == "TECNICA":
        print(f"[ROUTER] TECNICA (confidence={intent.confidence:.2f}) -> tech_agent")
        return "tech_agent"
    
    else:
        razon = (
            f"intent={intent.intent}, confidence={intent.confidence:.2f} "
            f"(umbral={CONFIDENCE_THRESHOLD})"
        )
        print(f"[ROUTER] {razon} -> fallback")
        return "fallback"


print(f"Funcion route_by_intent definida (umbral de confianza: {CONFIDENCE_THRESHOLD}).")

---

## 12. Construir el Grafo con LangGraph

Ahora ensamblamos todo. LangGraph nos permite definir:
- **Nodos**: Las funciones que procesan el estado.
- **Edges**: Las conexiones entre nodos.
- **Conditional edges**: Bifurcaciones dinamicas basadas en el estado.
- **Entry point**: Por donde entra el flujo.
- **END**: Nodo terminal.

In [ ]:
# =============================================================================
# Construir el grafo de routing
# =============================================================================

# 1. Crear el grafo con el tipo de estado
workflow = StateGraph(RouterState)

# 2. Agregar nodos
workflow.add_node("classify", classify_intent)        # Clasificador de intencion
workflow.add_node("product_agent", product_rag_agent)  # Agente de productos
workflow.add_node("tech_agent", tech_rag_agent)        # Agente tecnico
workflow.add_node("fallback", fallback_agent)          # Agente fallback

# 3. Definir punto de entrada
workflow.set_entry_point("classify")

# 4. Agregar edges condicionales desde el clasificador
workflow.add_conditional_edges(
    "classify",          # Nodo origen
    route_by_intent,     # Funcion que decide la ruta
    {                    # Mapeo: valor retornado -> nodo destino
        "product_agent": "product_agent",
        "tech_agent": "tech_agent",
        "fallback": "fallback",
    },
)

# 5. Los agentes terminan el flujo despues de responder
workflow.add_edge("product_agent", END)
workflow.add_edge("tech_agent", END)
workflow.add_edge("fallback", END)

# 6. Compilar el grafo
app = workflow.compile()

print("Grafo compilado exitosamente.")
print("Nodos: classify -> [product_agent | tech_agent | fallback] -> END")

---

## 13. Visualizar el Grafo

LangGraph puede generar una representacion visual del grafo. Esto es fundamental para entender y depurar el flujo.

In [ ]:
# =============================================================================
# Visualizar el grafo con Mermaid
# =============================================================================

try:
    # Intentar generar imagen PNG del grafo
    from IPython.display import Image, display
    
    graph_image = app.get_graph().draw_mermaid_png()
    display(Image(graph_image))
    print("Grafo renderizado como imagen.")
    
except Exception as e:
    print(f"No se pudo renderizar imagen ({e}).")
    print("Representacion textual del grafo (Mermaid):")
    print()
    print(app.get_graph().draw_mermaid())

---

## 14. Funcion auxiliar para ejecutar y mostrar resultados

Creamos una funcion helper que ejecuta una query por el grafo y muestra todos los detalles de forma legible.

In [ ]:
# =============================================================================
# Funcion auxiliar: ejecutar query y mostrar resultados detallados
# =============================================================================

def ejecutar_query(query: str, verbose: bool = True) -> dict:
    """Ejecuta una query por el grafo de routing y muestra resultados.
    
    Args:
        query: Consulta del usuario
        verbose: Si True, imprime detalles completos
    
    Returns:
        dict: Estado final del grafo
    """
    
    # Estado inicial
    estado_inicial = {
        "query": query,
        "intent": None,
        "retrieved_docs": [],
        "response": None,
        "route_taken": "",
        "steps_trace": [],
    }
    
    print("=" * 80)
    print(f"QUERY: {query}")
    print("=" * 80)
    
    # Ejecutar el grafo
    resultado = app.invoke(estado_inicial)
    
    if verbose and resultado.get("response"):
        resp = resultado["response"]
        print(f"\n{'─' * 60}")
        print(f"RESULTADO FINAL")
        print(f"{'─' * 60}")
        print(f"Ruta tomada:       {resultado['route_taken']}")
        print(f"Traza de pasos:    {' -> '.join(resultado['steps_trace'])}")
        print(f"Dominio respuesta: {resp.domain}")
        print(f"Confidence resp:   {resp.confidence}")
        print(f"Docs recuperados:  {len(resultado['retrieved_docs'])}")
        print(f"\nRESPUESTA:")
        print(resp.answer)
        if resp.sources:
            print(f"\nFUENTES ({len(resp.sources)}):")
            for i, src in enumerate(resp.sources):
                print(f"  [{i+1}] {src[:120]}..." if len(src) > 120 else f"  [{i+1}] {src}")
        print(f"\nSUGERENCIA: {resp.follow_up}")
    
    print("=" * 80)
    print()
    
    return resultado


print("Funcion ejecutar_query definida.")

---

## 15. Ejemplo 1: Query de Productos

Empezamos con una consulta clara de productos. Deberia clasificarse como **PRODUCTOS** con alta confianza y enrutarse al agente de productos.

Observa:
- Como el clasificador identifica la intencion.
- Que documentos recupera el agente de productos.
- La calidad de la respuesta con contexto especializado.

In [ ]:
# =============================================================================
# Ejemplo 1: Pregunta sobre precios de producto
# =============================================================================

resultado_1 = ejecutar_query(
    "Cuanto cuesta el plan Enterprise de DataSync y que incluye?"
)

---

## 16. Ejemplo 2: Query Tecnico

Ahora una consulta claramente tecnica. El clasificador deberia enrutarla al agente tecnico, que buscara en la coleccion `tecnica`.

Observa como los documentos recuperados son **completamente diferentes** a los del ejemplo anterior -- eso es el poder de tener colecciones separadas.

In [ ]:
# =============================================================================
# Ejemplo 2: Pregunta sobre procedimiento tecnico
# =============================================================================

resultado_2 = ejecutar_query(
    "Como hago rollback de un deploy en produccion?"
)

---

## 17. Ejemplo 3: Query Ambiguo

Este es el caso interesante. La query menciona "AI Assistant" (un producto) pero el problema es de "confianza baja" (que podria ser un tema de troubleshooting tecnico).

El clasificador tiene que decidir:
- Es una pregunta sobre el **producto** AI Assistant (como mejorarlo, que plan tiene)?
- Es una pregunta **tecnica** (como debugear el modelo, revisar embeddings)?

Presta atencion al score de confidence y al reasoning del clasificador.

In [ ]:
# =============================================================================
# Ejemplo 3: Query ambiguo que cruza dominios
# =============================================================================

resultado_3 = ejecutar_query(
    "Necesito ayuda con el AI Assistant, da respuestas con confianza baja"
)

---

## 18. Ejemplo 4: Query Fuera de Dominio

Cuando el usuario pregunta algo que no tiene nada que ver con NovaTech, el clasificador deberia retornar UNKNOWN y el sistema activar el fallback.

Esto es importante en produccion: un buen sistema RAG debe saber decir "no se" en lugar de inventar respuestas.

In [ ]:
# =============================================================================
# Ejemplo 4: Query completamente fuera de dominio
# =============================================================================

resultado_4 = ejecutar_query(
    "Cual es la receta del guacamole?"
)

---

## 19. Batch de Queries: Evaluacion Sistematica

Para evaluar el sistema de forma robusta, ejecutamos un **batch diverso** de queries y analizamos los patrones de clasificacion y routing.

In [ ]:
# =============================================================================
# Batch de queries diversas
# =============================================================================

queries_test = [
    # Productos (esperamos: PRODUCTOS con alta confianza)
    "Que diferencias hay entre el plan Starter y Professional de Analytics Pro?",
    "Cuantas consultas por mes incluye el plan Business de AI Assistant?",
    "Ofrecen descuentos por volumen para mas de 50 usuarios?",
    
    # Tecnica (esperamos: TECNICA con alta confianza)
    "Que hago si veo un error 429 rate limit exceeded?",
    "Como roto los secretos de JWT signing keys?",
    "Cual es el proceso de deploy de emergencia (hotfix)?",
    
    # Ambiguos (esperamos: clasificacion con confianza media)
    "El pipeline de DataSync esta fallando con timeout",
    "Necesito mejorar el rendimiento del sistema",
    
    # Fuera de dominio (esperamos: UNKNOWN)
    "Quien gano el mundial de futbol en 2022?",
    "Explicame la teoria de la relatividad",
]

# Ejecutar todas las queries y recolectar resultados
resultados_batch = []

for query in queries_test:
    resultado = ejecutar_query(query, verbose=False)  # verbose=False para no saturar el output
    
    # Extraer datos clave
    intent = resultado.get("intent")
    response = resultado.get("response")
    
    resultados_batch.append({
        "query": query[:60] + "..." if len(query) > 60 else query,
        "intent": intent.intent if intent else "N/A",
        "confidence": f"{intent.confidence:.2f}" if intent else "N/A",
        "route": resultado.get("route_taken", "N/A"),
        "answer_preview": response.answer[:80] + "..." if response else "N/A",
    })

print(f"\nBatch completado: {len(resultados_batch)} queries procesadas.")

In [ ]:
# =============================================================================
# Mostrar resultados del batch como tabla
# =============================================================================

df_resultados = pd.DataFrame(resultados_batch)

# Configurar pandas para mostrar texto completo
pd.set_option("display.max_colwidth", 80)
pd.set_option("display.width", 200)

print("\n" + "=" * 100)
print("TABLA DE RESULTADOS DEL BATCH")
print("=" * 100)
print(df_resultados.to_string(index=True))
print("\n")

# Resumen estadistico
print("RESUMEN DE ROUTING:")
print(df_resultados["route"].value_counts().to_string())

---

## 20. Visualizacion: Distribucion de Rutas

Un grafico nos permite ver rapidamente como se distribuyen las queries entre los dominios. En un sistema de produccion, monitorearias esta distribucion para detectar anomalias.

In [ ]:
# =============================================================================
# Grafico: distribucion de rutas tomadas
# =============================================================================

# Contar queries por ruta
distribucion = df_resultados["route"].value_counts()

# Colores por dominio
colores = {
    "product_agent": "#4CAF50",   # Verde para productos
    "tech_agent": "#2196F3",      # Azul para tecnica
    "fallback": "#FF9800",        # Naranja para fallback
}
colores_plot = [colores.get(ruta, "#9E9E9E") for ruta in distribucion.index]

# Crear figura con dos subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Subplot 1: Pie chart
ax1.pie(
    distribucion.values,
    labels=distribucion.index,
    autopct="%1.0f%%",
    colors=colores_plot,
    startangle=90,
    textprops={"fontsize": 11},
)
ax1.set_title("Distribucion de Rutas (Pie Chart)", fontsize=13, fontweight="bold")

# Subplot 2: Bar chart
bars = ax2.bar(distribucion.index, distribucion.values, color=colores_plot, edgecolor="black")
ax2.set_ylabel("Cantidad de queries", fontsize=11)
ax2.set_title("Distribucion de Rutas (Bar Chart)", fontsize=13, fontweight="bold")
ax2.set_ylim(0, max(distribucion.values) + 1)

# Agregar valor sobre cada barra
for bar, val in zip(bars, distribucion.values):
    ax2.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.1,
        str(val),
        ha="center",
        fontsize=12,
        fontweight="bold",
    )

plt.tight_layout()
plt.show()

print("Grafico generado.")

---

## 21. Ajuste de Umbral de Confianza

El umbral de confianza es un **hiperparametro** del sistema. Vamos a experimentar con tres valores:

| Umbral | Comportamiento esperado |
|--------|------------------------|
| 0.3    | Muy permisivo. Casi todo se enruta a un agente. Riesgo de respuestas incorrectas. |
| 0.6    | Balanceado. Acepta clasificaciones razonables. |
| 0.8    | Muy estricto. Muchas queries validas caen en fallback. |

Ejecutamos el **mismo batch de queries** con cada umbral y comparamos como cambian las decisiones de routing.

In [ ]:
# =============================================================================
# Experimento: impacto del umbral de confianza en routing
# =============================================================================

# Primero, recolectamos las clasificaciones (sin routing) para todas las queries
# Esto evita repetir las llamadas al clasificador
clasificaciones = []

for query in queries_test:
    estado_test = {
        "query": query,
        "intent": None,
        "retrieved_docs": [],
        "response": None,
        "route_taken": "",
        "steps_trace": [],
    }
    estado_clasificado = classify_intent(estado_test)
    clasificaciones.append({
        "query": query[:50] + "..." if len(query) > 50 else query,
        "intent": estado_clasificado["intent"].intent,
        "confidence": estado_clasificado["intent"].confidence,
    })

print(f"Clasificaciones recolectadas para {len(clasificaciones)} queries.")
print()

In [ ]:
# =============================================================================
# Simular routing con diferentes umbrales
# =============================================================================

umbrales = [0.3, 0.6, 0.8]
resultados_por_umbral = {}

for umbral in umbrales:
    rutas = []
    for c in clasificaciones:
        if c["confidence"] >= umbral and c["intent"] == "PRODUCTOS":
            rutas.append("product_agent")
        elif c["confidence"] >= umbral and c["intent"] == "TECNICA":
            rutas.append("tech_agent")
        else:
            rutas.append("fallback")
    resultados_por_umbral[umbral] = rutas

# Construir tabla comparativa
df_umbrales = pd.DataFrame({
    "query": [c["query"] for c in clasificaciones],
    "intent": [c["intent"] for c in clasificaciones],
    "confidence": [f"{c['confidence']:.2f}" for c in clasificaciones],
})

for umbral in umbrales:
    df_umbrales[f"ruta_{umbral}"] = resultados_por_umbral[umbral]

print("COMPARACION DE ROUTING CON DIFERENTES UMBRALES")
print("=" * 120)
print(df_umbrales.to_string(index=True))
print()

# Resumen: cuantas queries van a fallback con cada umbral
print("\nQUERIES EN FALLBACK POR UMBRAL:")
for umbral in umbrales:
    n_fallback = resultados_por_umbral[umbral].count("fallback")
    n_total = len(resultados_por_umbral[umbral])
    print(f"  Umbral {umbral}: {n_fallback}/{n_total} queries ({n_fallback/n_total*100:.0f}%) van a fallback")

In [ ]:
# =============================================================================
# Visualizacion: routing por umbral
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for i, umbral in enumerate(umbrales):
    rutas = resultados_por_umbral[umbral]
    conteo = pd.Series(rutas).value_counts()
    
    colores_barras = [colores.get(r, "#9E9E9E") for r in conteo.index]
    
    axes[i].bar(conteo.index, conteo.values, color=colores_barras, edgecolor="black")
    axes[i].set_title(f"Umbral = {umbral}", fontsize=13, fontweight="bold")
    axes[i].set_ylabel("Cantidad")
    axes[i].set_ylim(0, len(queries_test) + 1)
    
    # Valor sobre cada barra
    for bar_idx, (ruta_name, val) in enumerate(conteo.items()):
        axes[i].text(
            bar_idx, val + 0.15, str(val),
            ha="center", fontsize=12, fontweight="bold"
        )

plt.suptitle("Impacto del Umbral de Confianza en el Routing", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print("Observa como el umbral mas alto (0.8) envia mas queries al fallback.")
print("El balance entre precision y cobertura es un trade-off de ingenieria.")

---

## 22. Comparacion: Routing vs Single RAG

Para demostrar el valor del routing, implementamos un **RAG simple** que indexa TODOS los documentos en una sola coleccion y ejecutamos las mismas queries. Comparamos la calidad de las respuestas.

**Hipotesis**: El routing produce mejores respuestas porque:
1. Los documentos recuperados son mas relevantes (no hay contaminacion).
2. El prompt del agente esta especializado para el dominio.
3. Queries fuera de dominio se manejan explicitamente.

In [ ]:
# =============================================================================
# Crear RAG con coleccion unica (todos los docs mezclados)
# =============================================================================

# Combinar todos los chunks en una sola coleccion
todos_los_chunks = chunks_productos + chunks_tecnica
col_unica = crear_coleccion("todos_juntos", todos_los_chunks)

print(f"Coleccion unica creada: {col_unica.count()} documentos")
print(f"  (productos: {len(chunks_productos)} + tecnica: {len(chunks_tecnica)})")


# Prompt generico (no especializado)
SINGLE_RAG_PROMPT = """Eres un asistente de NovaTech Solutions.
Responde la pregunta del usuario basandote UNICAMENTE en el contexto proporcionado.
Si el contexto no contiene la informacion, dilo honestamente.
Responde en espanol de forma profesional.
"""


def single_rag_query(query: str) -> dict:
    """Ejecuta una query contra el RAG de coleccion unica (sin routing)."""
    
    # Recuperar documentos de la coleccion unica
    docs = buscar_documentos(col_unica, query, n_results=3)
    contexto = "\n\n---\n\n".join(docs)
    
    # Generar respuesta
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        temperature=0.2,
        messages=[
            {"role": "system", "content": SINGLE_RAG_PROMPT},
            {"role": "user", "content": f"CONTEXTO:\n{contexto}\n\nPREGUNTA:\n{query}"},
        ],
        response_format=RAGResponse,
    )
    
    return {
        "answer": response.choices[0].message.parsed.answer,
        "confidence": response.choices[0].message.parsed.confidence,
        "docs": docs,
    }


print("Single RAG configurado.")

In [ ]:
# =============================================================================
# Comparacion lado a lado: Routing RAG vs Single RAG
# =============================================================================

# Seleccionar queries representativas para comparar
queries_comparacion = [
    "Cuanto cuesta el plan Enterprise de DataSync?",            # Productos clara
    "Como hago rollback de emergencia en produccion?",          # Tecnica clara
    "El pipeline de DataSync esta fallando con timeout",        # Ambigua
    "Quien gano el mundial de futbol en 2022?",                # Fuera de dominio
]

print("COMPARACION: ROUTING RAG vs SINGLE RAG")
print("=" * 100)

for query in queries_comparacion:
    print(f"\nQUERY: {query}")
    print("-" * 80)
    
    # Routing RAG
    resultado_routing = ejecutar_query(query, verbose=False)
    resp_routing = resultado_routing.get("response")
    
    # Single RAG
    resultado_single = single_rag_query(query)
    
    print(f"\n  [ROUTING RAG]")
    print(f"  Ruta:       {resultado_routing.get('route_taken', 'N/A')}")
    print(f"  Confidence: {resp_routing.confidence if resp_routing else 'N/A'}")
    print(f"  Respuesta:  {resp_routing.answer[:200]}..." if resp_routing and len(resp_routing.answer) > 200 else f"  Respuesta:  {resp_routing.answer if resp_routing else 'N/A'}")
    
    print(f"\n  [SINGLE RAG]")
    print(f"  Confidence: {resultado_single['confidence']}")
    print(f"  Respuesta:  {resultado_single['answer'][:200]}..." if len(resultado_single['answer']) > 200 else f"  Respuesta:  {resultado_single['answer']}")
    
    print()

### Analisis de la comparacion

Puntos clave a observar:

1. **Queries claras de productos**: El routing deberia dar respuestas mas precisas porque recupera solo documentos de productos. El single RAG puede traer documentos tecnicos que mencionan el mismo producto pero no responden la pregunta.

2. **Queries claras tecnicas**: Similar al caso anterior pero con documentos tecnicos.

3. **Queries ambiguas**: Aqui el routing puede elegir un dominio (o ir a fallback), mientras que el single RAG mezcla documentos de ambos dominios sin criterio.

4. **Queries fuera de dominio**: El routing tiene un fallback explicito. El single RAG intenta responder con documentos irrelevantes, dando respuestas potencialmente enganosas.

---

## 23. Combinando Routing + Prompt Chaining

En un sistema de produccion real, no usas routing **o** prompt chaining -- usas **ambos**.

### Arquitectura combinada

```
User Query
    |
    v
[Intent Classifier] -----> Routing Layer
    |
    +-----> PRODUCTOS -----> [Agente Especializado con Chaining Interno]
    |                           |
    |                           +---> Paso 1: Reformular query para el dominio
    |                           +---> Paso 2: Retrieve documentos
    |                           +---> Paso 3: Generar respuesta borrador
    |                           +---> Paso 4: Validar contra fuentes
    |                           +---> Paso 5: Formatear respuesta final
    |
    +-----> TECNICA -----> [Agente Especializado con Chaining Interno]
    |                           |
    |                           +---> (mismo pipeline, diferente prompt/contexto)
    |
    +-----> FALLBACK -----> Mensaje de clarificacion
```

### Lo que esto significa para el proyecto

El **Notebook 05 (proyecto integrador)** combina todos estos patrones:
- Routing para clasificar y delegar.
- Prompt chaining dentro de cada agente especializado.
- Evaluacion y optimizacion del pipeline completo.

Este notebook (04) te da las herramientas de routing. El siguiente paso es integrarlas en un sistema completo.

---

## 24. Errores Comunes

### 1. No incluir ejemplos en el prompt del clasificador
Sin ejemplos concretos de cada dominio, el LLM tiende a clasificar demasiadas queries como UNKNOWN. Los few-shot examples son criticos para la calidad de la clasificacion.

### 2. Umbral de confianza estatico en produccion
Un umbral fijo (como 0.6) funciona para un demo, pero en produccion necesitas monitorear la distribucion de confidence scores y ajustar dinamicamente. Si el 80% de tus queries llegan con confidence > 0.95, tu umbral de 0.6 no esta discriminando nada util.

### 3. No tener fallback (o un fallback que miente)
Algunos sistemas omiten el fallback y fuerzan siempre una respuesta. Esto genera alucinaciones. Un buen fallback dice "no se" y guia al usuario. Peor que no responder es responder mal.

### 4. Colecciones no balanceadas
Si la coleccion de productos tiene 500 chunks y la tecnica tiene 20, el agente tecnico tendra retrieval pobre. Asegurate de que cada dominio tenga documentacion suficiente. Si un dominio es "pobre", mejor fusionarlo con otro.

### 5. Ignorar queries ambiguas en las metricas
Las queries que el clasificador envia a fallback por baja confianza son **senales valiosas**. Analizalas periodicamente: muchas queries ambiguas en un mismo tema pueden indicar que necesitas un nuevo dominio o mejorar la documentacion existente.

---

## 25. Checklist de Comprension

Antes de pasar al siguiente notebook, deberias poder responder estas preguntas:

- [ ] **Por que separamos las colecciones por dominio** en lugar de indexar todo junto? Que problema resuelve?

- [ ] **Que pasa si el umbral de confianza es demasiado alto** (ej. 0.95)? Y si es demasiado bajo (ej. 0.1)?

- [ ] **Como agregarias un tercer dominio** (ej. BILLING) al grafo? Que nodos, edges y prompts necesitas?

- [ ] **Que ventaja tiene el structured output** (Pydantic) sobre parsear texto libre del LLM para la clasificacion?

- [ ] **En que escenario real de AI Engineering** aplicarias este patron de routing manana mismo?

---

**Siguiente paso**: Notebook 05 - Proyecto integrador que combina routing + prompt chaining + evaluacion.